# **Лабораторная работа №5. Визуализация данных с использованием библиотек Python.Интерактивная визуализация с Plotly**

In [1]:
import plotly
import plotly.graph_objs as go
import plotly.graph_objects as go
from plotly.offline import iplot
import plotly.express as px
from plotly.subplots import make_subplots

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data = pd.read_csv('/content/drive/MyDrive/bfeed.csv')

In [4]:
data['education']=data['yschool'].map(lambda x: 'only school (or less)' if x < 13 else 'higher education')

In [5]:
data.head(5)

,Unnamed: 0,duration,delta,race,poverty,smoke,alcohol,agemth,ybirth,yschool,pc3mth,education
0,1,16,1,1,0,0,1,24,82,14,0,higher education
1,2,1,1,1,0,1,0,26,85,12,0,only school (or less)
2,3,4,0,1,0,0,0,25,85,12,0,only school (or less)
3,4,3,1,1,0,1,1,21,85,9,0,only school (or less)
4,5,36,1,1,0,1,0,22,82,12,0,only school (or less)


**Описание данных**

* **duration** - Продолжительность грудного вскармливания (нед.). *Количественная*.

* **delta** - Индикатор завершения грудного вскармливания (1=да, 0=нет). *Категориальный*

* **race** - Раса матери (1=белая, 2=черная, 3=другая). *Категориальный*

* **poverty** - Мать в бедности (1=да, 0=нет). *Категориальный*

* **smoke** - Мать курила при рождении ребенка (1=да, 0=нет). *Категориальный*

* **alcohol** - Мать употребляла алкоголь при рождении ребенка (1=да, 0=нет). *Категориальный*

* **agemth** - Возраст матери при рождении ребенка. *Количественный*

* **ybirth** - Год рождения ребенка. *Количественный*

* **yschool** - Уровень образования матери (годы обучения). *Количественный*

* **pc3mth** - Дородовой уход после 3-го месяца (1=да, 0=нет). *Категориальный*

**Дородовой уход является одной из форм профилактической медицины.  Он предоставляется в форме медицинских осмотров, включая пренатальный скрининг и диагностику, а также рекомендации о том, как вести здоровый образ жизни и предоставление медицинской информации, такой как физиологические изменения матери во время беременности, биологические изменения и пренатальное питание.*

## 1. «Ящик с усами»

In [6]:
fig = px.box(data, x="education", y="ybirth", color="race")
fig.show()

Подтвердили вывод полученный в 4ой лабораторной.

In [7]:
data3=pd.DataFrame(data[['agemth', 'duration']])
data3 = data3[data3['duration']<150]

In [8]:
po = []
for i in data3['agemth'].sort_values().unique():
  po.append(go.Box(y=data3[data3['agemth']==i].duration, name=str(i)))

iplot(po)

## 2. Гистограммы

In [9]:
fig = make_subplots(rows=2, cols=2)

trace0 = go.Histogram(x=data['duration'], name="duration")
trace1 = go.Histogram(x=data['agemth'], name="agemth")
trace2 = go.Histogram(x=data['yschool'], name="yschool")
trace3 = go.Histogram(x=data['ybirth'], name="ybirth")

fig.append_trace(trace0, 1, 1)
fig.append_trace(trace1, 1, 2)
fig.append_trace(trace2, 2, 1)
fig.append_trace(trace3, 2, 2)

fig.update_layout(bargap=0.2)
fig.update_layout(title_text="Гистограммы для количественных признаков")
fig.show()

In [10]:
fig = px.histogram(data, x="education", y="yschool", color="education", pattern_shape="poverty", histfunc="count", text_auto=True)
fig.show()

Среди женщин закончивших только школу ~23% находяться в тяжелом материальном положении.

Среди же матерей с высшим образованием бедных всего ~7,5%.

## 3. Круговые диаграммы

In [11]:
labels_race = ["White", "Black", "Another"]
labels = ['No', 'Yes']

# Create subplots: use 'domain' type for Pie subplot
fig = make_subplots(rows=2, cols=3, specs=[[{'type':'domain'}, {'type':'domain'}, {'type':'domain'}],
                                           [{'type':'domain'}, {'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=labels_race, values=data['race'].value_counts().sort_index(), name="race"),
              1, 1)
fig.add_trace(go.Pie(labels=labels, values=data['poverty'].value_counts().sort_index(), name="poverty"),
              1, 2)
fig.add_trace(go.Pie(labels=labels, values=data['education'].value_counts(), name="education"),
              1, 3)
fig.add_trace(go.Pie(labels=labels, values=data['smoke'].value_counts().sort_index(), name="smoke"),
              2, 1)
fig.add_trace(go.Pie(labels=labels, values=data['alcohol'].value_counts().sort_index(), name="alcohol"),
              2, 2)
fig.add_trace(go.Pie(labels=labels, values=data['pc3mth'].value_counts().sort_index(), name="pc3mth"),
              2, 3)


# Use `hole` to create a donut-like pie chart
fig.update_traces(hole=.4, hoverinfo="label+percent+name")

fig.update_layout(
    title_text="Круговые диаграммы для качественных признаков",
    # Add annotations in the center of the donut pies.
    annotations=[dict(text='Race', x=0.135, y=0.8245, font_size=15, showarrow=False),
                 dict(text='Poverty', x=0.5, y=0.8245, font_size=15, showarrow=False),
                 dict(text='Education', x=0.88, y=0.8245, font_size=15, showarrow=False),
                 dict(text='Smoke', x=0.13, y=0.18, font_size=15, showarrow=False),
                 dict(text='Alcohol', x=0.5, y=0.18, font_size=15, showarrow=False),
                 dict(text='Prenatal care', x=0.89, y=0.18, font_size=15, showarrow=False)],
                 title_pad_l=100)
fig.show()

In [12]:
age = data['agemth'].value_counts().sort_index()

fig = px.pie(values=age.values, names=age.index, hole=.2)
fig.update_traces(marker_line_width=2, marker_line_color="white")
fig.update_layout(title_text="Распределение возраста матерей")
fig.show()

**Выводы:** С помощью библиотеки Plotly подтвердили выводы полученные в предыдущих лабораторных по-новому взлянув на данные. Интерактивная визуализация нагляднее, динамичнее и удобнее, чем статическая, следовательно, потенциально может содержать гораздо больше информации, что позволяет быстро понять ключевые моменты.